<a href="https://colab.research.google.com/github/sujeet311/Quantuma_Boosting/blob/main/quantum_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

pip install dwave.system

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from sklearn.impute import SimpleImputer
#from qboost import QBoostClassifier,QboostPlus
import time
import numpy as np

In [ ]:
from sklearn.metrics import make_scorer, f1_score, recall_score, precision_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

Import the train and test data set.

In [ ]:
from numpy import loadtxt
X_train = loadtxt('/content/drive/MyDrive/FINALCODE/Data/traintest/X_train.csv',delimiter=',')
y_train = loadtxt('/content/drive/MyDrive/FINALCODE/Data/traintest/y_train.csv',delimiter=',')
X_test = loadtxt('/content/drive/MyDrive/FINALCODE/Data/traintest/X_test.csv',delimiter=',')
y_test = loadtxt('/content/drive/MyDrive/FINALCODE/Data/traintest/y_test.csv',delimiter=',')

binaary spin (+1) and (-1) in order to convert into the format for Ising model

In [ ]:
y_train = (2* y_train )-1
y_test = (2* y_test) -1

In [ ]:
# scaling data

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Import the trained model on same data. the model selected on the basis of there performance score. Since Adaboost, weighted average voting, from classical is selected and qboost from qpu based selected for the ensemble.

In [ ]:
import joblib
 # qboost
QBC= joblib.load('/content/drive/MyDrive/FINALCODE/Model/Qbc.pkl' )
ada = joblib.load('/content/drive/MyDrive/FINALCODE/Model/ada.pkl' )   #  Adaboost
wt_avg=joblib.load('/content/drive/MyDrive/FINALCODE/Model/Wt_avg.pkl' )  #weighted average


In [ ]:
y_pred = QBC.predict(X_test)
QBC_accu = accuracy_score(y_test, y_pred)
print("Test accurracy",QBC_accu)


Test accurracy 0.7235460056419183


APi key for solver. in this case free solver DW_2000Q_6 is selected

In [ ]:
# define sampler
endpoint = 'https://cloud.dwavesys.com/sapi'
token = 'DEV-a71a4e9dc32bb8967a50c5470614d84b8ca0e842'
solver = 'DW_2000Q_6'
dwave_sampler = DWaveSampler(endpoint=endpoint, token=token, solver=solver)
emb_sampler = EmbeddingComposite(dwave_sampler)
lmd = 1.0

In [ ]:
# define parameters used in this function
NUM_READS = 1000
NUM_WEAK_CLASSIFIERS = 50
TREE_DEPTH = 2
DW_PARAMS = {'num_reads': NUM_READS,
                 'auto_scale': True,
                 'num_spin_reversal_transforms': 10,
                 'postprocess': 'optimization',
                 }
emb_sampler = EmbeddingComposite(dwave_sampler)

In [ ]:
class QboostPlus(object):
    """
    Only for Classifiers
    Quantum boost existing (weak) classifiers
    """

    def __init__(self, weak_classifier_list):
        self.estimators_ = weak_classifier_list
        self.n_estimators = len(self.estimators_)
        self.estimator_weights = np.ones(self.n_estimators) #estimator weights will be binary (Dwave output)

    def fit(self, X, y, sampler, lmd=0.2, **kwargs):

        n_data = len(X)
        # step 1: create QUBO
        hij = []
        for h in self.estimators_:
            hij.append(h.predict(X))

        hij = np.array(hij)
        # scale hij to [-1/N, 1/N]
        hij = 1. * hij / self.n_estimators
        return hij

        ## Create QUBO
        qii = n_data * 1. / (self.n_estimators ** 2) + lmd - 2 * np.dot(hij, y)
        qij = np.dot(hij, hij.T)
        Q = dict()
        Q.update(dict(((k, k), v) for (k, v) in enumerate(qii)))
        for i in range(self.n_estimators):
            for j in range(i + 1, self.n_estimators):
                Q[(i, j)] = qij[i, j]

        # step 3: optimize QUBO
        res = sampler.sample_qubo(Q, **kwargs)
        samples = np.array([[samp[k] for k in range(self.n_estimators)] for samp in res])

        # take the optimal solution as estimator weights
        self.estimator_weights = samples[0]

    def predict(self, X):

        n_data = len(X)
        T = 0
        y = np.zeros(n_data)
        for i, h in enumerate(self.estimators_):
            y0 = self.estimator_weights[i] * h.predict(X)  # prediction of weak classifier
            y += y0
            T += np.sum(y0)

        y = np.sign(y - T / (n_data*self.n_estimators))

        return y

In [ ]:
ensemble = QboostPlus([wt_avg,ada,QBC])
t=time.time()
ensemble.fit(X_train, y_train, emb_sampler, lmd=lmd, **DW_PARAMS)
print("Weight assign to each model=",ensemble.estimator_weights)
y_pred = ensemble.predict(X_test)
print("Train Accurracy=",accuracy_score(y_train, ensemble.predict(X_train)))

ensemble_accu = accuracy_score(y_test, y_pred)
print("Test accurracy",ensemble_accu)
# confusion matrix

print("Confusion matrix :",confusion_matrix(y_test, y_pred))
# classification report
target_names = ['0','1']
print(classification_report(y_test, y_pred,target_names=target_names))
elapsed_ab = time.time()-t
print("Time taken:",elapsed_ab)


Weight assign to each model= [1. 1. 1.]
Train Accurracy= 0.9628629279711312
Test accurracy 0.8916031450693236
Confusion matrix : [[48544  9486]
 [ 3156 55441]]
              precision    recall  f1-score   support

           0       0.94      0.84      0.88     58030
           1       0.85      0.95      0.90     58597

    accuracy                           0.89    116627
   macro avg       0.90      0.89      0.89    116627
weighted avg       0.90      0.89      0.89    116627

Time taken: 30.622539520263672


In [ ]:
from joblib import Parallel, delayed
import joblib
# Save the model as a pickle in a file
joblib.dump(ensemble, '/content/drive/MyDrive/FINALCODE/Model/Qensemble.pkl')

['/content/drive/MyDrive/FINALCODE/Model/Qensemble.pkl']